# 事前準備
## ライブラリのインポート

In [21]:
from gurobipy import Model, quicksum, GRB
import pandas as pd
import pickle

## 事前に用意されたデータの読み込み
読み込んでいる`logi_data.dump`ファイルは、同ディレクトリ内にある`make_logi_data.ipynb`ファイルで作成しています。

In [30]:
with open('logi_data.dump', 'rb') as f:
    Cust = pickle.load(f)                #顧客の集合
    DC = pickle.load(f)                   #流通センターの集合
    Route = pickle.load(f)              #配送ルートの集合
    CT = pickle.load(f)                   #センタータイプの集合
    transport = pickle.load(f)       #輸送費
    delivery = pickle.load(f)          #配送費
    dc_run = pickle.load(f)           #流通センター維持費
    dc_stock = pickle.load(f)       #流通センター在庫費
    dc_ub = pickle.load(f)            #流通センター保管容量
    dc_new = pickle.load(f)         #流通センター新規契約費
    dc_cancel = pickle.load(f)     #流通センター解約費
    dc_disposal = pickle.load(f)  #流通センター在庫廃棄費
    cust_stock = pickle.load(f)   #販社在庫費
    cust_out = pickle.load(f)       #販社品切れ費

## 計画期間の決定

In [31]:
T = 8                                             #モデルの計画期の決定
Period = list(range(1, T+1))      #計画期間のリスト

## demand.csvファイルを読み込み

In [24]:
df = pd.read_csv('demand.csv', header=None)
demand = {(j,t):df[t-1][j-1] for j in Cust for t in Period}

# モデルの構築
## モデルの作成

In [25]:
model = Model()

## 変数の追加

In [27]:
X, x, y, sD, sC, z, w, o, d = {}, {}, {}, {}, {}, {}, {}, {}, {}

#期tにおける流通センターiへの輸送量X[i,t]を追加
for i in DC:
    for t in Period:
        X[i,t] = model.addVar(vtype="I", name=f'X[{i},{t}]')

#期tにおける流通センターiから販社jへの配送量x[i,j,t]を追加
for i,j in Route:
    for t in Period:
        x[i,j,t] = model.addVar(vtype="I", name=f'x[{i},{j},{t}]')

#期tにおいて地点iにセンタータイプctの流通センターを運用するかどうかのバイナリ変数y[i,ct,t]を追加
for i in DC:
    for ct in CT:
        y[i,ct,0] = model.addVar(vtype="B", ub=0, name=f'y[{i},{ct},{0}]')
        for t in Period:
            y[i,ct,t] = model.addVar(vtype="B", name=f'y[{i},{ct},{t}]')

#期tにおける流通センターiの在庫量sD[i,t]を追加
for i in DC:
    sD[i,0] = model.addVar(vtype="I", ub=0, name=f'sD[{i},{t}]')
    for t in Period:
        sD[i,t] = model.addVar(vtype="I", name=f'sD[{i},{t}]')
        d[i,t] = model.addVar(vtype="I", name=f'd[{i},{t}]')

#期tにおける販社jの在庫量sC[j,t]を追加
for j in Cust:
    sC[j,0] = model.addVar(vtype="I", ub=0, name=f'sC[{j},{t}]')
    for t in Period:
        sC[j,t] = model.addVar(vtype="I", name=f'sC[{j},{t}]')

#期tにおいて地点iにセンタータイプctの流通センターを新しく建てるかどうかのバイナリ変数z[i,ct,t]を追加
for i in DC:
    for ct in CT:
        for t in Period:
            z[i,ct,t] = model.addVar(vtype="B", name=f'z[{i},{ct},{t}]')

#期tにおいて地点iにセンタータイプctの流通センターを解約するかどうかのバイナリ変数w[i,ct,t]を追加
for i in DC:
    for ct in CT:
        for t in Period:
            w[i,ct,t] = model.addVar(vtype="B", name=f'w[{i},{ct},{t}]')

#期tにおける販社jの品切れ量o[j,t]を追加
for j in Cust:
    for t in Period:
        o[j,t] = model.addVar(vtype="I", name=f'o[{j},{t}]')

#期tにおける流通センターiの在庫量廃棄量d[i,t]を追加
for i in DC:
    for t in Period:
        d[i,t] = model.addVar(vtype="I", name=f'd[{i},{t}]')

        
model.update()

## 制約の追加

In [28]:
Cust_Demand_Cons, DC_Running_Cons, CT_Only_Cons, DC_Flow_Cons, DC_UB_Cons, DC_Connect_Cons = {}, {}, {}, {}, {}, {}

for j in Cust:
    for t in Period:
        Cust_Demand_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in DC) + sC[j,t-1]
            ==
            demand[j,t] + sC[j,t] - o[j,t]
        )

for i,j in Route:
    for t in Period:
        DC_Running_Cons[i,j] = model.addConstr(
            x[i,j,t]
            <=
            quicksum(demand[j,t_] for t_ in Period) * quicksum(y[i,ct,t] for ct in CT)
        )

for i in DC:
    for t in Period:
        CT_Only_Cons[i,t] = model.addConstr(
            quicksum(y[i,ct,t] for ct in CT)
            <=
            1
        )

for i in DC:
    for t in Period:
        DC_Flow_Cons = model.addConstr(
            X[i,t] + sD[i,t-1]
            ==
            quicksum(x[i,j,t] for j in Cust) + sD[i,t] + d[i,t]
        )

for i in DC:
    for t in Period:
        DC_UB_Cons[i,t] = model.addConstr(
            X[i,t] + sD[i,t-1]
            <=
            quicksum(dc_ub[ct] * y[i,ct,t] for ct in  CT)
        )

for i in DC:
    for ct in CT:
        for t in Period:
            DC_Connect_Cons[i,ct,t] = model.addConstr(
                y[i,ct,t] - y[i,ct,t-1]
                ==
                z[i,ct,t] - w[i,ct,t]
            )
    
model.update()

## 目的関数の設定

In [17]:
model.setObjective(
    quicksum(transport[i] * X[i,t] for i in DC for t in Period) +
    quicksum(delivery[i,j] * x[i,j,t] for i,j in Route for t in Period) +
    quicksum(dc_run[i,ct] * y[i,ct,t] for i in DC for ct in CT for t in Period) +
    quicksum(dc_stock * sD[i,t] for i in DC for t in Period) +
    quicksum(cust_stock * sC[j,t] for j in Cust for t in Period) +
    quicksum(dc_new * z[i,ct,t] for i in DC for ct in CT for t in Period) +
    quicksum(dc_cancel * w[i,ct,t] for i in DC for ct in CT for t in Period) +
    quicksum(cust_out * o[j,t] for j in Cust for t in Period) +
    quicksum(dc_disposal * d[i,t] for i in DC for t in Period)
    ,GRB.MINIMIZE
)

model.update()

## 求解

In [29]:
model.optimize()

Optimize a model with 1760 rows, 5610 columns and 11176 nonzeros
Variable types: 0 continuous, 5610 integer (2750 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


# 最適解の可視化

In [19]:
for v in model.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

X[7,1] 80.0
X[7,2] 80.0
X[7,3] 80.0
X[7,4] 72.0
X[7,5] 78.0
X[7,6] 71.0
X[7,7] 80.0
X[7,8] 80.0
X[9,5] 59.0
X[9,6] 80.0
X[9,7] 80.0
X[9,8] 80.0
X[10,1] 80.0
X[10,2] 80.0
X[10,3] 80.0
X[10,4] 80.0
X[10,5] 80.0
X[10,6] 80.0
X[10,7] 80.0
X[10,8] 80.0
X[11,6] 57.0
X[11,7] 68.0
X[11,8] 80.0
x[11,11,6] 32.0
x[11,11,7] 40.0
x[11,11,8] 41.0
x[11,5,6] 25.0
x[11,5,7] 14.0
x[11,5,8] 19.0
x[7,6,1] 20.0
x[7,6,2] 16.0
x[7,6,3] 17.0
x[7,6,4] 10.0
x[7,6,5] 22.0
x[7,6,6] 33.0
x[7,6,7] 36.0
x[7,6,8] 30.0
x[9,9,5] 30.0
x[9,9,6] 47.0
x[9,9,7] 39.0
x[9,9,8] 34.0
x[7,8,1] 16.0
x[7,8,2] 20.0
x[7,8,3] 23.0
x[7,8,4] 10.0
x[7,8,6] 2.0
x[10,3,1] 12.0
x[10,3,2] 15.0
x[10,3,3] 7.0
x[10,3,4] 11.0
x[10,3,5] 6.0
x[10,3,6] 12.0
x[10,3,7] 25.0
x[10,3,8] 16.0
x[7,2,1] 12.0
x[7,2,2] 9.0
x[7,2,3] 6.0
x[7,2,4] 14.0
x[7,2,5] 15.0
x[10,9,1] 8.0
x[10,9,2] 17.0
x[10,9,3] 18.0
x[11,4,8] 9.0
x[10,4,1] 8.0
x[10,4,2] 8.0
x[10,4,3] 8.0
x[10,4,4] 9.0
x[10,4,5] 8.0
x[10,4,6] 14.0
x[10,4,7] 7.0
x[10,4,8] 3.0
x[7,1,1] 5.0
x[7,1,3] 5.0


In [20]:
model.ObjVal

93384.8